# Unit 2: Popularity Recommendations

In this section we build a recommender that sorts items by popularity as of the number of ratings they received. As a result we return the $N$ most popular items as recommendations.

In [ ]:
from typing import Dict, List

import numpy as np
import pandas as pd
from scipy.stats import spearmanr

In [ ]:
# `Dataset` is just a wrapper for the MovieLens training data
from recsys_training.data import Dataset, genres

In [ ]:
ml100k_ratings_filepath = '../data/raw/ml-100k/u.data'
ml100k_item_filepath = '../data/raw/ml-100k/u.item'

## Load Data

We load the dataset with $100`000$ ratings and split it $4:1$ into train and test set.

(**Remark**: We do not focus on proper hyperparameter search within this tutorial and therefore do not generate a separate validation dataset)

In [ ]:
data = Dataset(ml100k_ratings_filepath)
data.rating_split(train_size=0.8, seed=42)

In [ ]:
items = pd.read_csv(ml100k_item_filepath, sep='|', header=None,
                    names=['item', 'title', 'release', 'video_release', 'imdb_url']+genres,
                    engine='python')

In [ ]:
data.train_ratings

In [ ]:
data.test_ratings

Build a Mapping from user id to its item ratings. We will need this later.

In [ ]:
user_ratings = data.get_user_ratings()

Show up to 20 user ratings for the first user

In [ ]:
user = 1
list(user_ratings[user].items())[:20]

## Popularity Ranking

How do we define _popularity_? It turns out that there can be different things justifying the popularity of content:
- **pure count**: simply count the number of ratings or interactions an item received regardless of their quality
- **positive count**: only count the number of ratings or interactions that we assume reflect preference towards items, e.g. ratings above user mean ratings
- **time-dependency**: despite evergreen stars items may also be popular for a limited time only - how can we account for this?

**Remark**: Popularity ranking entails no personalization. We obtain a single popularity ranking of items which is independent from the user and serve the same top-$N$ items to every user.

### Popularity based on simple Interaction Counts

![](parrot.png)

**Task**: Infer the item popularity order from training ratings as an array with items in descending order of popularity.

In [ ]:
item_popularity = data.train_ratings.item.value_counts()

In [ ]:
item_popularity

In [ ]:
item_order = item_popularity.values

In [ ]:
item_order

What are the most popular movies?

In [ ]:
top_movie_ids = item_order[:5]
items[items['item'].isin(top_movie_ids)][['item', 'title']]

### Popularity based on positive Interaction Counts

We assume that the the mean rating for each user is the threshold above which movies are regarded as favorable and below which movies are deemed as bad.

1. compute that user mean rating for each user.
2. remove all ratings that fall below this threshold.
3. apply the process above to the remaining ratings.

In [ ]:
user_mean_ratings = data.train_ratings[['user', 'rating']].groupby('user')
user_mean_ratings = user_mean_ratings.mean().reset_index()
user_mean_ratings.rename(columns={'rating': 'user_mean_rating'},
                         inplace=True)

In [ ]:
user_mean_ratings

In [ ]:
positive_train_ratings = data.train_ratings.merge(user_mean_ratings,
                                                  on='user',
                                                  how='left')

In [ ]:
keep_ratings = (positive_train_ratings['rating'] >= positive_train_ratings['user_mean_rating'])

In [ ]:
positive_train_ratings = positive_train_ratings[keep_ratings]
positive_train_ratings.drop(columns='user_mean_rating', inplace=True)

In [ ]:
positive_train_ratings

In [ ]:
item_popularity_positive = positive_train_ratings.item.value_counts()

In [ ]:
item_popularity_positive

In [ ]:
item_order_positive = item_popularity.index.values

In [ ]:
items[items['item'].isin(item_order_positive[:5])][['item', 'title']]

#### How strong do both orderings correlate with each other?

Check spearman rank correlation between both orderings to quantify the distortion in ordering.

In [ ]:
joint_counts = [[item_popularity.loc[item], item_popularity_positive[item]]
                for item in np.intersect1d(item_popularity_positive.index.values,
                                           item_popularity.index.values)]
joint_counts = np.array(joint_counts)

In [ ]:
joint_counts

In [ ]:
spearmanr(joint_counts)

### Using Popularity Ordering for top-$N$ Recommendations

Now, we can produce recommendations from our popularity ordering.

![](parrot.png)

**Task**: Write a method `get_recommendation` that returns the top-$N$ items without any known positives, i.e. items the user has already viewed.

In [ ]:
def get_recommendations(user: int,
                        user_ratings: dict,
                        item_popularity_order: np.array,
                        N: int) -> List[int]:
    known_positives = None
    recommendations = None
    
    return recommendations

Try it ...

In [ ]:
get_recommendations(1, user_ratings, item_order, 10)

## Evaluating the Relevance of Recommendations

In [ ]:
def get_relevant_items(test_ratings: pd.DataFrame) -> Dict[int, List[int]]:
    """
    returns {user: [items]} as a list of relevant items per user
    for all users found in the test dataset
    """
    relevant_items = test_ratings[['user', 'item']]
    relevant_items = relevant_items.groupby('user')
    relevant_items = {user: relevant_items.get_group(user)['item'].values
                      for user in relevant_items.groups.keys()}

    return relevant_items

In [ ]:
relevant_items = get_relevant_items(data.test_ratings)

In [ ]:
relevant_items[1]

### $Precision@10$

Now, we can compute the intersection between the top-$N$ recommended items and the items each user interacted with. Ideally, we want every recommendation to be a hit, i.e. an item the user consumed. In this case the size of intersections is $N$ given $N$ recommendations which is a precision of 100% = $\frac{N}{N}$.

We compute the so called $Precision@N$ for every user and take the mean over all. The resulting metric is called _mean average precision at N_ or short $MAP@N$.

![](parrot.png)

**Task:** Compute the $MAP@N$ for popularity recommendations

In [ ]:
def get_precision(users: List[int], user_ratings: Dict[int, Dict[int, float]],
                  item_order: np.array, N: int) -> Dict[int, float]:
    
    pass
    
    return prec_at_N

Try it ...

In [ ]:
N = 10
users = relevant_items.keys()

In [ ]:
prec_at_N = get_precision(users, user_ratings, item_order, N)

In [ ]:
np.mean(list(prec_at_N.values()))